In [7]:
import numpy as np
import pandas as pd
from datetime import datetime 
from datetime import timedelta
import csv 
import matplotlib.pyplot as plt
from datetime import date
from datetime import time
import calendar
import locale
import seaborn as sns

In [8]:
def convertTimeStamp(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p') #Accepts time string, converts it to datetime object.
    return time

In [9]:
#convert seconds to weeks, days, hours, minutes, seconds

intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

In [10]:
def elapsedInMonths(filepath):
    """councilTime accepts a csv file and returns a dataframe which displays the average elapsed time between created date 
        and update date for service requests in each distinct council district.
        
        Data source: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2017/d4vt-q4t5
        """
    
    dataframe = pd.read_csv(filepath)
    
    
    #filter dateset to CreatedDate and UpdatedDate columns
    created_df = dataframe['CreatedDate'].dropna().apply(convertTimeStamp, 0) #apply convertTimeStamp to CreatedDate
    updated_df = dataframe['UpdatedDate'].dropna().apply(convertTimeStamp, 0) #apply convertTimeStamp to UpdatedDate
    
    
    #create elapsed time and elapsed seconds columns and insert into dataframe
    elapsed_df = updated_df - created_df #calculate the difference in the values in CreatedDate and UpdatedDate
    dataframe.insert(3, "ElapsedTime", elapsed_df) #insert column ElapsedTime with dataframe elapsed_df into dataframe dataframe
    elapsed_seconds = dataframe['ElapsedTime'].dt.total_seconds() #convert ElapsedTime values to seconds
    dataframe.insert(4, 'ElapsedTime in Seconds', elapsed_seconds)
    
    
    #convert CreatedDate and UpdatedDate to datetime objects
    created_df = pd.to_datetime(dataframe['CreatedDate'], format='%m/%d/%Y %I:%M:%S %p')
    updated_df = pd.to_datetime(dataframe['UpdatedDate'], format='%m/%d/%Y %I:%M:%S %p')
    
    
    df4 = dataframe[['CreatedDate', 'UpdatedDate', 'ElapsedTime', 'ElapsedTime in Seconds']] #create a dataframe df4 with dates and elapsed time columns from dataframe
    df4 = df4.sort_values(by = 'CreatedDate')
        
    
    #create a dataframe of the months of CreatedDate rows
    for row in range(0, len(created_df)-1):
        created_df[row] = created_df[row].month
        
    
    #create dataframes for each unique month with ElapsedTime column
    monthsList = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    for num in range(0,11):
        month = monthsList[num]
        print('Elapsed Times in', month)
        
        #group CreatedDate by month
        is_month =  created_df==num+1
        df4_month = df4[is_month]
        
        #print columns CreatedDate, UpdatedDate, ElapsedTime, ElapsedTime in Seconds
        month_time = df4_month[['CreatedDate', 'UpdatedDate', 'ElapsedTime']]
        print(month_time)

        #find average elapsed time for each month
        elapsed_avg = df4_month['ElapsedTime in Seconds'].mean()
        elapsed_avg = display_time(elapsed_avg, 5)

        print('\nAverage Elapsed Time in', monthsList[num],':', elapsed_avg)
        print('\n\n')
        
    

In [11]:
elapsedInMonths(r"C:\Users\hanaa\Downloads\MyLA311_Service_Request_Data_2017.csv")

C:\Users\hanaa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Elapsed Times in January


C:\Users\hanaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


                    CreatedDate             UpdatedDate      ElapsedTime
1131554  01/01/2017 01:01:00 AM  01/04/2017 05:33:00 PM  3 days 16:32:00
1131330  01/01/2017 01:01:00 PM  01/03/2017 08:54:00 AM  1 days 19:53:00
1131328  01/01/2017 01:01:00 PM  01/03/2017 08:52:00 AM  1 days 19:51:00
1131329  01/01/2017 01:01:00 PM  01/09/2017 01:19:00 PM  8 days 00:18:00
1131327  01/01/2017 01:02:00 PM  01/17/2017 12:29:00 PM 15 days 23:27:00
...                         ...                     ...              ...
1047066  01/31/2017 12:58:00 PM  02/01/2017 12:04:00 PM  0 days 23:06:00
1047058  01/31/2017 12:59:00 PM  02/06/2017 12:13:00 PM  5 days 23:14:00
1047057  01/31/2017 12:59:00 PM  02/01/2017 01:18:00 PM  1 days 00:19:00
1047060  01/31/2017 12:59:00 PM  04/13/2017 09:34:00 AM 71 days 20:35:00
1047059  01/31/2017 12:59:00 PM  02/02/2017 05:04:00 PM  2 days 04:05:00

[86566 rows x 3 columns]

Average Elapsed Time in January : 4.0 days, 23.0 hours, 2.0 minutes, 7.0 seconds



Elapsed Times